In [2]:
import pandas as pd

In [3]:
data = pd.read_stata(".\data\hdf_online_service & review time.dta")

In [30]:
data.head()

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge
0,1001121010,16095.0,6482734404,8886209813,20220906 08:14,1.0,,NaN,Master only (1)
1,1001121010,376906.0,6496218600,8883707471,20220604 16:20,1.0,,NaN,Master only (1)
2,1001121010,376908.0,6496218610,8883707483,20220604 16:20,1.0,,NaN,Master only (1)
3,1001121010,376909.0,6496218611,8883707486,20220604 16:20,1.0,,NaN,Master only (1)
4,1001121010,376910.0,6496218612,8883707485,20220604 16:20,1.0,,NaN,Master only (1)


# EDA

## Calculate the number of online reviews written by each individual patient til a particular timestamp

In [5]:
data["patient_id"].value_counts()

patient_id
24697.0     24
211030.0    23
27371.0     22
32336.0     20
32317.0     19
            ..
523355.0     1
521259.0     1
507550.0     1
481531.0     1
544106.0     1
Name: count, Length: 564735, dtype: int64

### Remove patient that only appears once

In [40]:
repeated_patient = data[data.groupby('patient_id').patient_id.transform('count') > 1]

In [41]:
repeated_patient["patient_id"].value_counts()

patient_id
24697.0     24
211030.0    23
27371.0     22
32336.0     20
32317.0     19
            ..
404760.0     2
405638.0     2
406283.0     2
407343.0     2
500949.0     2
Name: count, Length: 54624, dtype: int64

### Group by patient and sort by service time ascending

In [42]:
repeated_patient_sort_by_time = repeated_patient.sort_values(['patient_id','service_time'], ascending=True).groupby(['patient_id'])

In [43]:
repeated_patient_sort_by_time.head()

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge
68436,14392,16.0,6416067266,8883449494,20220524 11:49,1.0,,NaN,Master only (1)
68437,14392,16.0,6416067266,8884265165,20220627 11:39,1.0,20220709 13:14,1.0,Matched (3)
68438,14392,16.0,6416067266,8885268088,20220802 15:04,1.0,,NaN,Master only (1)
68439,14392,16.0,6416067266,8886243454,20220907 13:05,1.0,,NaN,Master only (1)
263355,259551,18.0,6416070618,8879858313,20211226 14:24,1.0,,NaN,Master only (1)
...,...,...,...,...,...,...,...,...,...
578072,6964511427,564247.0,6499497397,8887519837,20221029 15:58,1.0,,NaN,Master only (1)
667847,958822725,564470.0,6499511440,8887531973,20221030 09:41,0.0,,NaN,Master only (1)
667848,958822725,564470.0,6499511440,8887535847,20221030 12:33,0.0,,NaN,Master only (1)
402803,5033859274,564637.0,6499530599,8887554549,20221031 11:37,0.0,,NaN,Master only (1)


In [44]:
repeated_patient_sort_by_time["on_review_time"].value_counts()

patient_id  on_review_time
16.0                          3
            20220709 13:14    1
18.0                          3
23.0                          2
24.0                          2
                             ..
564120.0                      2
564134.0                      2
564247.0                      2
564470.0                      2
564637.0                      2
Name: count, Length: 71727, dtype: int64

### Count

In [45]:
repeated_patient.columns

Index(['doctor_id', 'patient_id', 'service_list', 'service_id', 'service_time',
       'text', 'on_review_time', 'on_review_rating', '_merge'],
      dtype='object')

In [46]:
# Count number of reviews
new_df_with_count = pd.DataFrame(columns=['doctor_id', 'patient_id', 'service_list', 'service_id', 'service_time',
       'text', 'on_review_time', 'on_review_rating', '_merge', 'no_patient_review'])
for patient in repeated_patient_sort_by_time["patient_id"].unique():
    counts = []
    patient_id = patient[0]
    temp_data = repeated_patient_sort_by_time.get_group((patient_id,))
    temp_data = temp_data.reset_index()
    first_time = None
    for i, (j, row) in enumerate(temp_data.iterrows()):
        if i == 0:
            first_time = row["service_time"]

        current_time = row["service_time"]
        count = ((temp_data["on_review_time"].notnull()) & (temp_data["on_review_time"] > first_time) & (temp_data["on_review_time"] <= current_time)).sum()
        counts.append(count)
    temp_data["no_patient_review"] = counts
    new_df_with_count = pd.concat([new_df_with_count, temp_data], axis = 0)

C:\Users\nguyetpvt\AppData\Local\Temp\ipykernel_23568\2148733059.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df_with_count = pd.concat([new_df_with_count, temp_data], axis = 0)
C:\Users\nguyetpvt\AppData\Local\Temp\ipykernel_23568\2148733059.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df_with_count = pd.concat([new_df_with_count, temp_data], axis = 0)


In [47]:
new_df_with_count

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge,no_patient_review,index
0,14392,16.0,6416067266,8883449494,20220524 11:49,1.0,,NaN,Master only (1),0,68436.0
1,14392,16.0,6416067266,8884265165,20220627 11:39,1.0,20220709 13:14,1.0,Matched (3),0,68437.0
2,14392,16.0,6416067266,8885268088,20220802 15:04,1.0,,NaN,Master only (1),1,68438.0
3,14392,16.0,6416067266,8886243454,20220907 13:05,1.0,,NaN,Master only (1),1,68439.0
0,259551,18.0,6416070618,8879858313,20211226 14:24,1.0,,NaN,Master only (1),0,263355.0
...,...,...,...,...,...,...,...,...,...,...,...
1,6964511427,564247.0,6499497397,8887519837,20221029 15:58,1.0,,NaN,Master only (1),0,578072.0
0,958822725,564470.0,6499511440,8887531973,20221030 09:41,0.0,,NaN,Master only (1),0,667847.0
1,958822725,564470.0,6499511440,8887535847,20221030 12:33,0.0,,NaN,Master only (1),0,667848.0
0,5033859274,564637.0,6499530599,8887554549,20221031 11:37,0.0,,NaN,Master only (1),0,402803.0


In [49]:
new_df_with_count = new_df_with_count.drop(columns=["index"], axis=1)

In [50]:
new_df_with_count

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge,no_patient_review
0,14392,16.0,6416067266,8883449494,20220524 11:49,1.0,,NaN,Master only (1),0
1,14392,16.0,6416067266,8884265165,20220627 11:39,1.0,20220709 13:14,1.0,Matched (3),0
2,14392,16.0,6416067266,8885268088,20220802 15:04,1.0,,NaN,Master only (1),1
3,14392,16.0,6416067266,8886243454,20220907 13:05,1.0,,NaN,Master only (1),1
0,259551,18.0,6416070618,8879858313,20211226 14:24,1.0,,NaN,Master only (1),0
...,...,...,...,...,...,...,...,...,...,...
1,6964511427,564247.0,6499497397,8887519837,20221029 15:58,1.0,,NaN,Master only (1),0
0,958822725,564470.0,6499511440,8887531973,20221030 09:41,0.0,,NaN,Master only (1),0
1,958822725,564470.0,6499511440,8887535847,20221030 12:33,0.0,,NaN,Master only (1),0
0,5033859274,564637.0,6499530599,8887554549,20221031 11:37,0.0,,NaN,Master only (1),0


In [53]:
# Count number of positive review
counts = []
for patient in repeated_patient_sort_by_time["patient_id"].unique():
    patient_id = patient[0]
    temp_data = repeated_patient_sort_by_time.get_group((patient_id,))
    temp_data = temp_data.reset_index()
    first_time = None
    for i, (j, row) in enumerate(temp_data.iterrows()):
        if i == 0:
            first_time = row["service_time"]

        current_time = row["service_time"]
        count = ((temp_data["on_review_time"].notnull()) & (temp_data["on_review_time"] > first_time) & (temp_data["on_review_time"] <= current_time) & (temp_data["on_review_rating"] == 1.0)).sum()
        counts.append(count)

In [54]:
len(counts)

138735

In [55]:
new_df_with_count["no_patient_positive_review"] = counts

In [56]:
new_df_with_count

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge,no_patient_review,no_patient_positive_review
0,14392,16.0,6416067266,8883449494,20220524 11:49,1.0,,NaN,Master only (1),0,0
1,14392,16.0,6416067266,8884265165,20220627 11:39,1.0,20220709 13:14,1.0,Matched (3),0,0
2,14392,16.0,6416067266,8885268088,20220802 15:04,1.0,,NaN,Master only (1),1,1
3,14392,16.0,6416067266,8886243454,20220907 13:05,1.0,,NaN,Master only (1),1,1
0,259551,18.0,6416070618,8879858313,20211226 14:24,1.0,,NaN,Master only (1),0,0
...,...,...,...,...,...,...,...,...,...,...,...
1,6964511427,564247.0,6499497397,8887519837,20221029 15:58,1.0,,NaN,Master only (1),0,0
0,958822725,564470.0,6499511440,8887531973,20221030 09:41,0.0,,NaN,Master only (1),0,0
1,958822725,564470.0,6499511440,8887535847,20221030 12:33,0.0,,NaN,Master only (1),0,0
0,5033859274,564637.0,6499530599,8887554549,20221031 11:37,0.0,,NaN,Master only (1),0,0


In [57]:
new_df_with_count.to_csv("./data/no_patient_review_til_current_time.csv", encoding="utf-8")

## Number of review from other patients for each doctor-patient pair til a particular timestamp

### Remove patient that only appears once

In [85]:
repeated_patient = data[data.groupby('patient_id').patient_id.transform('count') > 1]

In [88]:
repeated_patient

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge
10,1001121010,546797.0,6498421461,8886258206,20220907 22:17,0.0,20220912 15:57,1.0,Matched (3)
11,1001121010,546797.0,6498421461,8886494757,20220917 21:55,0.0,,NaN,Master only (1)
24,1002307215,17702.0,6483904331,8879185449,20211130 13:06,1.0,,NaN,Master only (1)
25,1002307215,17702.0,6483904331,8881159216,20220218 13:46,1.0,,NaN,Master only (1)
31,1002307215,35282.0,6490787876,8881327406,20220224 14:13,1.0,,NaN,Master only (1)
...,...,...,...,...,...,...,...,...,...
672722,995582909,488750.0,6497528996,8886174138,20220904 16:58,1.0,,NaN,Master only (1)
672759,995596798,206199.0,6493532265,8880491693,20220120 08:26,1.0,,NaN,Master only (1)
672760,995596798,206199.0,6493532265,8881732523,20220310 20:50,1.0,,NaN,Master only (1)
672806,995596798,500949.0,6497670802,8885405898,20220807 14:08,0.0,,NaN,Master only (1)


### Group by doctor then sort by service time ascending

In [97]:
repeated_patient_gb_doctor_sort_by_time = repeated_patient.sort_values(['doctor_id', 'service_time', 'patient_id'], ascending=True).groupby(['doctor_id'])

In [98]:
repeated_patient_gb_doctor_sort_by_time.head()

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge
10,1001121010,546797.0,6498421461,8886258206,20220907 22:17,0.0,20220912 15:57,1.0,Matched (3)
11,1001121010,546797.0,6498421461,8886494757,20220917 21:55,0.0,,NaN,Master only (1)
24,1002307215,17702.0,6483904331,8879185449,20211130 13:06,1.0,,NaN,Master only (1)
65,1002307215,188053.0,6493332485,8880258444,20220111 11:23,1.0,20220301 08:06,1.0,Matched (3)
25,1002307215,17702.0,6483904331,8881159216,20220218 13:46,1.0,,NaN,Master only (1)
...,...,...,...,...,...,...,...,...,...
672722,995582909,488750.0,6497528996,8886174138,20220904 16:58,1.0,,NaN,Master only (1)
672759,995596798,206199.0,6493532265,8880491693,20220120 08:26,1.0,,NaN,Master only (1)
672760,995596798,206199.0,6493532265,8881732523,20220310 20:50,1.0,,NaN,Master only (1)
672806,995596798,500949.0,6497670802,8885405898,20220807 14:08,0.0,,NaN,Master only (1)


### Count

In [99]:
# Count number of reviews
new_df_with_count = pd.DataFrame(columns=['doctor_id', 'patient_id', 'service_list', 'service_id', 'service_time',
       'text', 'on_review_time', 'on_review_rating', '_merge', 'no_other_patient_review'])

for doctor in repeated_patient_gb_doctor_sort_by_time["doctor_id"].unique():
    doctor_id = doctor[0]
    temp_data = repeated_patient_gb_doctor_sort_by_time.get_group((doctor_id,))
    temp_data = temp_data.reset_index()

    counts = []
    first_time = None
    for i, (j, row) in enumerate(temp_data.iterrows()):
        if i == 0:
            first_time = row["service_time"]

        current_time = row["service_time"]
        count = ((temp_data["on_review_time"].notnull()) & (temp_data["on_review_time"] > first_time) & (temp_data["on_review_time"] <= current_time) & (temp_data["patient_id"] != row["patient_id"])).sum()
        counts.append(count)

    temp_data["no_other_patient_review"] = counts
    new_df_with_count = pd.concat([new_df_with_count, temp_data], axis = 0)

C:\Users\nguyetpvt\AppData\Local\Temp\ipykernel_23568\2344622601.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df_with_count = pd.concat([new_df_with_count, temp_data], axis = 0)
C:\Users\nguyetpvt\AppData\Local\Temp\ipykernel_23568\2344622601.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df_with_count = pd.concat([new_df_with_count, temp_data], axis = 0)


In [100]:
new_df_with_count

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge,no_other_patient_review,index
0,1001121010,546797.0,6498421461,8886258206,20220907 22:17,0.0,20220912 15:57,1.0,Matched (3),0,10.0
1,1001121010,546797.0,6498421461,8886494757,20220917 21:55,0.0,,NaN,Master only (1),0,11.0
0,1002307215,17702.0,6483904331,8879185449,20211130 13:06,1.0,,NaN,Master only (1),0,24.0
1,1002307215,188053.0,6493332485,8880258444,20220111 11:23,1.0,20220301 08:06,1.0,Matched (3),0,65.0
2,1002307215,17702.0,6483904331,8881159216,20220218 13:46,1.0,,NaN,Master only (1),0,25.0
...,...,...,...,...,...,...,...,...,...,...,...
3,995582909,488750.0,6497528996,8886174138,20220904 16:58,1.0,,NaN,Master only (1),0,672722.0
0,995596798,206199.0,6493532265,8880491693,20220120 08:26,1.0,,NaN,Master only (1),0,672759.0
1,995596798,206199.0,6493532265,8881732523,20220310 20:50,1.0,,NaN,Master only (1),0,672760.0
2,995596798,500949.0,6497670802,8885405898,20220807 14:08,0.0,,NaN,Master only (1),0,672806.0


In [101]:
new_df_with_count[new_df_with_count["no_other_patient_review"] != 0]

,doctor_id,patient_id,service_list,service_id,service_time,text,on_review_time,on_review_rating,_merge,no_other_patient_review,index
5,1002307215,272770.0,6494374400,8881472218,20220301 18:39,1.0,20220316 20:27,1.0,Matched (3),1,76.0
6,1002307215,277763.0,6494434716,8881542184,20220303 22:29,1.0,20220306 15:03,1.0,Matched (3),1,78.0
7,1002307215,309121.0,6494910421,8882103105,20220325 19:14,1.0,,NaN,Master only (1),3,88.0
8,1002307215,337136.0,6495480104,8882802791,20220425 19:19,1.0,20220427 22:42,1.0,Matched (3),3,99.0
9,1002307215,339192.0,6495525080,8882858146,20220427 22:26,1.0,,NaN,Master only (1),3,102.0
...,...,...,...,...,...,...,...,...,...,...,...
104,988757719,551731.0,6498774167,8886667871,20220924 10:29,1.0,,NaN,Master only (1),11,672566.0
105,988757719,36975.0,6491099166,8886817996,20220929 22:12,1.0,,NaN,Master only (1),11,671873.0
106,988757719,551731.0,6498774167,8886876388,20221002 15:14,1.0,,NaN,Master only (1),11,672567.0
107,988757719,551731.0,6498774167,8887041893,20221010 12:36,1.0,,NaN,Master only (1),11,672568.0


In [104]:
# Count number of positive review
counts = []
for doctor in repeated_patient_gb_doctor_sort_by_time["doctor_id"].unique():
    doctor_id = doctor[0]
    temp_data = repeated_patient_gb_doctor_sort_by_time.get_group((doctor_id,))
    temp_data = temp_data.reset_index()
    
    first_time = None
    for i, (j, row) in enumerate(temp_data.iterrows()):
        if i == 0:
            first_time = row["service_time"]

        current_time = row["service_time"]
        count = ((temp_data["on_review_time"].notnull()) & (temp_data["on_review_time"] > first_time) & (temp_data["on_review_time"] <= current_time) & (temp_data["patient_id"] != row["patient_id"]) & (temp_data["on_review_rating"] == 1.0)).sum()
        counts.append(count)

In [105]:
len(counts)

138735

In [106]:
new_df_with_count = new_df_with_count.drop(columns=["index"], axis=1)

In [ ]:
new_df_with_count["no_other_patient_positive_review"] = counts

In [113]:
new_df_with_count.to_csv("./data/no_other_patient_review_til_current_time.csv", encoding="utf-8")